# Incremental Load

Exercise:

### Incremental Extract, Load and Transform
We expect to get new data every month which we will incrementally load.

In [ ]:
import snowflake.snowpark as snp
import json
import getpass


with open('creds.json') as f:
    data = json.load(f)
    connection_parameters = {
      'account': data['account'],
      'user': data['username'],
      'password': data['password'], #getpass.getpass(),
      'role': data['role'],
      'warehouse': data['warehouse']}

session = snp.Session.builder.configs(connection_parameters).create()

In [ ]:
from citibike_ml import elt as ELT

import snowflake.snowpark as snp

import pandas as pd
from datetime import datetime
import os
import uuid 

start = datetime.now()
print("Start Time =", start.strftime("%H:%M:%S"))

download_base_url = 'https://s3.amazonaws.com/tripdata/'

project_db_name = 'CITIBIKEML'
project_schema_name = 'DEMO'
project_db_schema = str(project_db_name)+'.'+str(project_schema_name)

load_table_name = str(project_db_schema)+'.'+'RAW_'
trips_table_name = str(project_db_schema)+'.'+'TRIPS'

_ = session.sql('USE DATABASE '+str(project_db_name)).collect()
_ = session.sql('USE SCHEMA '+str(project_db_schema)).collect()

load_stage_name = 'load_stage'
session.sql('CREATE OR REPLACE TEMPORARY STAGE '+str(load_stage_name)).collect()

In [ ]:
file_name_end2 = '202102-citibike-tripdata.csv.zip'
file_name_end1 = '201402-citibike-tripdata.zip'

files_to_download = [file_name_end2]

In [ ]:
files_to_download

In [ ]:
%%time
load_stage_name, files_to_load = ELT.extract_trips_to_stage(session=session, 
                                                            files_to_download=files_to_download, 
                                                            download_base_url=download_base_url, 
                                                            load_stage_name=load_stage_name)

In [ ]:
%%time
stage_table_names = ELT.load_trips_to_raw(session, files_to_load, load_stage_name, load_table_name)

In [ ]:
%%time
trips_table_name = ELT.transform_trips(session, stage_table_names, trips_table_name)

In [ ]:
%%writefile citibike_ml/ingest.py

def incremental_elt(session, load_stage_name, files_to_download, download_base_url, load_table_name, trips_table_name) -> str:
    from citibike_ml import elt as ELT

    load_stage_name, files_to_load = ELT.extract_trips_to_stage(session=session, 
                                                                files_to_download=files_to_download, 
                                                                download_base_url=download_base_url, 
                                                                load_stage_name=load_stage_name)
    stage_table_names = ELT.load_trips_to_raw(session, 
                                              files_to_load=files_to_load, 
                                              load_stage_name=load_stage_name, 
                                              load_table_name=load_table_name)
    
    trips_table_name = ELT.transform_trips(session=session, 
                                           stage_table_names=stage_table_names, 
                                           trips_table_name=trips_table_name)
    return trips_table_name



In [ ]:
session.close()

In [ ]:
#TODO: How to avoid duplicates?